<a href="https://colab.research.google.com/github/pyrated03/Few-Shot-Learning-IIT-Bombay/blob/main/Noise_Introduction_RNNP(SUB_3_part_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import torch.optim as optim


import gc
import numpy as np
import pandas as pd
import cv2
import os
import time
import os.path as osp
import pickle
from PIL import Image
import PIL
import random
import math
from torch.optim import lr_scheduler


import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC

In [ ]:
# Load the pretrained model
model_vec = models.resnet18(pretrained=True)
# Use the model object to select the desired layer
layer = model_vec._modules.get('avgpool')


In [ ]:
# Set model to evaluation mode
model_vec.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [ ]:
def get_vector(img):
    # 1. Load the image with Pillow library
    # img = Image.open(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
      my_embedding.copy_(o.data.reshape(o.data.size(1)))
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model_vec(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding

In [ ]:
class CUB2002011_ds(Dataset):
    def __init__(self, data_dir, mode="train", noise=0.000001):
        assert mode in ["train", "test"] #Make sure the mode is one of the two
        super(CUB2002011_ds, self).__init__() #Inheritance dataset __init__()
        train_test_split = pd.read_csv(
            f"{data_dir}/train_test_split.txt",
            sep=" ",
            names=["img_id", "is_train"],
        )
        img_paths = pd.read_csv(
            f"{data_dir}/images.txt", sep=" ", names=["img_id", "img_path"]
        )  #Read all image paths
        labels = pd.read_csv(
            f"{data_dir}/image_class_labels.txt",
            sep=" ",
            names=["img_id", "img_label"],
        )
        self.noise = noise
        self.data_dir = osp.join(data_dir, "images")
        self.mode = 1 if mode == "train" else 0  #Choose whether it is 1 training mode or test mode 0
        self.img_path = img_paths.loc[
            train_test_split["is_train"] == self.mode, "img_path"
        ].values  #Get the corresponding path in the corresponding mode
        self.label = labels.loc[
            train_test_split["is_train"] == self.mode, "img_label"
        ].values #Get the label in the corresponding mode
        img_sz = 84
        if mode == 'test':
            self.transform = transforms.Compose([
                transforms.Resize((img_sz, img_sz)),
                transforms.ToTensor(),
                # transforms.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225))
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((img_sz, img_sz)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(30),
                transforms.ToTensor(),
                # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        self.y = self.label

    def __len__(self):
        return self.label.shape[0] #The size of the entire data set in the corresponding mode

    def __getitem__(self, idx):
        img = self.transform(
            Image.open(f"{self.data_dir}/{self.img_path[idx]}").convert("RGB")
        ) #Open the path of the image in the corresponding folder of the data set and convert the image data to RGB
        label = self.label[idx]
        label = label-1
        # NOISE INTRODUCTION
        if self.mode == 1:
          if idx % (math.ceil(1/self.noise)) == 0:
            ran = label
            while(ran == label):
              ran = random.randint(0,199)
            
            label = ran
        return img, label


In [ ]:
CUB_ds = CUB2002011_ds("/content/drive/MyDrive/IITB Internship/CUB_200_2011", "train")#,0.2)
train_CUB = torch.utils.data.DataLoader(CUB_ds, batch_size=54, shuffle=False, drop_last=True)


In [ ]:
train_CUB_length = len(train_CUB)*54

In [ ]:
CUB_dvec_master = {}
X_orig = []
for images, labels in train_CUB:
  for i in range(54):
    labels = np.array(labels)
    x_i = (get_vector(transforms.ToPILImage()(images[i]))).numpy()
    X_orig.append(x_i)
    if labels[i] not in CUB_dvec_master:
      CUB_dvec_master[labels[i]] = []
      CUB_dvec_master[labels[i]].append(x_i)
    else:
      CUB_dvec_master[labels[i]].append(x_i)
    # print(type(images))

In [ ]:
def augment(L,orig_data, alpha, beta):
  for i in range(beta):
    im1, im2 = random.sample(orig_data, 2)
    L.append(im1 + (alpha)*im2)
          
  return L


In [ ]:
Unlabelled_Aug = []
for i in range(200):
  Unlabelled_Aug = augment(Unlabelled_Aug, CUB_dvec_master[i], 0.25, 5)

Merging to make the feature matrix to train


In [ ]:
X = []
for i in range(200):
  X = X + CUB_dvec_master[i]

X_aug = X + Unlabelled_Aug
Y = []
for i in range(200):
  list_labels = [i] * len(CUB_dvec_master[i])
  Y = Y + list_labels
# random.shuffle(Y)
list_unlabelled = [-1] * len(Unlabelled_Aug)
Y_aug = Y + list_unlabelled

In [ ]:
temp = list(zip(X_aug, Y_aug))
random.shuffle(temp)
X_fin, Y_fin = zip(*temp)

In [ ]:
# svc = SVC(probability=True, gamma="auto")
# lr = LogisticRegression(max_iter=1000)
model_svc = SVC(kernel='rbf', 
                probability=True, # Need to enable to be able to use predict_proba
                C=1.5, # default = 1.0
                gamma='scale', # default = 'scale',
                random_state=0
               )
# self_training_model = SelfTrainingClassifier(lr, threshold=0.6) #criterion = "k_best")
self_training_model = SelfTrainingClassifier(base_estimator=model_svc, # An estimator object implementing fit and predict_proba.
                                             threshold=0.8, # default=0.75, The decision threshold for use with criterion='threshold'. Should be in [0, 1).
                                             criterion='threshold', # {‘threshold’, ‘k_best’}, default=’threshold’, The selection criterion used to select which labels to add to the training set. If 'threshold', pseudo-labels with prediction probabilities above threshold are added to the dataset. If 'k_best', the k_best pseudo-labels with highest prediction probabilities are added to the dataset.
                                             #k_best=50, # default=10, The amount of samples to add in each iteration. Only used when criterion='k_best'.
                                             max_iter=None, # default=10, Maximum number of iterations allowed. Should be greater than or equal to 0. If it is None, the classifier will continue to predict labels until no new pseudo-labels are added, or all unlabeled samples have been labeled.
                                             verbose=True # default=False, Verbosity prints some information after each iteration
                                            )
clf_ST = self_training_model.fit(X_fin, Y_fin)

End of iteration 1, added 6 new labels.


In [ ]:
Y_temp = clf_ST.predict(X)
accuracy_score(Y, Y_temp)

0.8293293293293293

In [ ]:
Y_orig = clf_ST.predict(X_orig)

In [ ]:
y_orig_ar = np.array(Y_orig) + 1
# file = open("/content/drive/MyDrive/IITB Internship/CUB_200_2011/Y_After_Noise.txt", "w+")
 
# Saving the array in a text file
# content = str(Y_orig)
# file.write(content)
# file.close()
np.savetxt("/content/drive/MyDrive/IITB Internship/CUB_200_2011/Y_After_Noise.txt", y_orig_ar)